<a href="https://colab.research.google.com/github/mbtiongson1/Overengineered-A-Algorithm/blob/main/Overengineered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AI 201 Programming Assignment 1 (PA1)

**Marcus Rafael B. Tiongson** <br>
2013-59474 <br>
February 20, 2026 <br>






#Introduction: 8-Puzzle

**Heuristic search** is used to implement the **A(*) algorithm** to solve the  8-Puzzle problem.

##The Problem Statement

The best path to the goal will be determined using the following heuristics:

  a) **Number of Tiles** in the wrong position ($h_1$)

  b) **Manhattan Distance** ($h_2$)

  c) **Nilsson's Sequence Score** ($h_3$) based on [Stack Overflow](https://stackoverflow.com/questions/10584788/can-anyone-explain-nilssons-sequence-score-in-8-puzzle-more-clearly/10607141#10607141)


**Additional Notes:**
- **Sequence of Moves** will be shown on matrices.
- **Number of Moves** $g(n)$ will be calculated.
- **Heuristic Score** $h_i(n)$ will be calculated.
- **Number of Nodes Expanded** will also be shown.
- **Manhattan Distance** $P(n)$ and **Nilsson's Sequence Score** $S(n)$ will be calculated and shown if applicable.


##Initializing the input `astar_in.txt`

The input file should be on this format:
```
start
2 1 6
4 * 8
7 5 3
goal
1 2 3
8 * 4
7 6 5
```
Edit the file /content/astar_in.txt in the notebook to get started.

Then, **run** the code snippet below to initialize and print the current `astar_in.txt` file.

In [179]:
#test print input file
_INPUT_PATH = "astar_in.txt"
with open(_INPUT_PATH, "r") as f:
    print(f.read())

start
2 1 6
4 * 8
7 5 3
goal
1 2 3
8 * 4
7 6 5


##Initializing `start` and `goal` matrices
In order to work with the data, the `start` and `goal` matrices need to be parsed into arrays from the `astar_in.txt` file.

**Run** the code below to parse the file into `start` and `goal` matrices.

**Expand** the cells to test if matrices have been created. These will be needed in the algorithm calculations.

In [180]:
#Change the input to a matrix (start) and matrix (goal). Below is the sample matrices.
start=[[2, 1, 6],
     [4, 0, 8],
     [7, 5, 3]]
goal=[[1, 2, 3],
     [8, 0, 4],
     [7, 6, 5]]
#* is 0 to minimize errors... this will be brought back later
#Parsing the input file:
def _parse_matrix_block(lines):
    matrix = []
    for line in lines:
        row = []
        for tok in line.strip().split():
            if tok == '*': #change * to 0
                row.append(0)
            else:
                row.append(int(tok))
        if len(row) != 3:
            raise ValueError(f"Expected 3 values per row, got {len(row)} in line: {line!r}")
        matrix.append(row)
    if len(matrix) != 3:
        raise ValueError(f"Expected 3 rows, got {len(matrix)}")
    return matrix

def parse_start_goal_from_file(path: str) -> Tuple[List[List[int]], List[List[int]]]:
    with open(path, 'r', encoding='utf-8') as f:
        raw_lines = [line.rstrip('\n') for line in f if line.strip() != '']

    # Normalize to lower for headers but keep original rows
    lowered = [ln.lower() for ln in raw_lines]

    try:
        start_idx = lowered.index('start')
    except ValueError:
        raise ValueError("Missing 'start'")
    try:
        goal_idx = lowered.index('goal')
    except ValueError:
        raise ValueError("Missing 'goal'")

    if goal_idx <= start_idx:
        raise ValueError("'goal' must appear after 'start'.")

    # Extract exactly 3 lines after each header for 3x3 matrices
    if start_idx + 3 >= len(raw_lines):
        raise ValueError("Not enough lines to read the 3x3 'start' matrix.")
    start_block = raw_lines[start_idx + 1 : start_idx + 4]

    if goal_idx + 3 >= len(raw_lines):
        raise ValueError("Not enough lines to read the 3x3 'goal' matrix.")
    goal_block = raw_lines[goal_idx + 1 : goal_idx + 4]

    start_matrix = _parse_matrix_block(start_block)
    goal_matrix = _parse_matrix_block(goal_block)
    return start_matrix, goal_matrix

def Start():
    s, _ = parse_start_goal_from_file(_INPUT_PATH)
    return s

def Goal():
    _, g = parse_start_goal_from_file(_INPUT_PATH)
    return g


###Test Print the `start` and `goal` matrices
**Run** the code below to check if the matrices have been parsed. These will be used for the algorithm.

In [181]:
#test print matrices
import numpy as np

print("Start matrix:")
print(np.array(Start()))

print("\nGoal matrix:")
print(np.array(Goal()))


Start matrix:
[[2 1 6]
 [4 0 8]
 [7 5 3]]

Goal matrix:
[[1 2 3]
 [8 0 4]
 [7 6 5]]


Algorithm Calculations: Cost and Heuristic Functions
==========
The A* algorithm will be used. The `score` function $f(n)$ will be calculated as follows: <br>
$$f(n) = h(n) + g(n)$$

Where $g(n) =$ `cost` and $h(n) =$ `heuristic`

The heuristic function $h(n)$ will have three versions, namely:
1. $h_1(n)$ - the **Misplaced Tiles** (`misplaced`) or number of tiles in the wrong direction
2. $h_2(n)$ - the **Manhattan Distance** ($P(n)$ or `distance`) or the sum of distance of each tile from the goal position
3. $h_3(n)$ - the `3` times **Nilsson's Sequence Score** ($S(n)$ or `nscore`) plus the Manhattan Distance ($P(n)$ or `distance`)

**Expand** the cells below to see all calculations for $g(n)$ and $h_i(n)$.

Cost Function $g(n)$
-------

The cost function $g(n)$ will be calculated as the cost of movement of the path taken. In this case, this is simply the number of moves so far.

$$g_{i+1}(n) = g_i(n) + 1$$

$g(n) = $number of moves $=$ `cost`

Misplaced Tiles Heuristic $h_1(n)$
-------

The heuristic function will be the total number of tiles in the wrong position. This is straightforward and the sum of tiles will be calculated as follows for every matrix:

$h_1(n) =$ count of $n$ for all $n$ in the wrong position $=$ `misplaced`

$$f_1(n) = g(n) + h_1(n)$$

$f_1(n) =$ `cost + misplaced`

One move in any direction `up`, `down`, `left`, or `right` should have their own calculation $f_1(n)$.

Manhattan Distance Heuristic $h_2(n)$
-------

The heuristic function will be the total sum of distance (or Manhattan Distance) away from the goal position. This is calculated by setting a value for each tile or node and calculating how many steps minimum away from the goal position it is.

$h_2(n) =$ sum of distances $x_n =$ `distance`

where $x_n = x_n(goal) - x_n(current)$

$$f_2(n) = g(n) + h_2(n) = P(n)$$
$P(n) =$ `cost + distance`

One move in any direction `up`, `down`, `left`, or `right` should have their own calculation $h_2(n)$.

Nilsson Score Heuristic $h_3(n)$
-------

The heuristic function tallies a score epening on the tiles' position.
- `center`: The center tile has a score of `1`.
- `2 * sequence`: The outside tiles each have a score:
  - if the clockwise tile beside it is **in sequence**, score is `0`.
  - if the clockwise tile next to it is **NOT in sequence**, score is `2`.

The sum $S(n)$ is determined to be `nscore` multiplied by `3`.

$$3S(n) = 3*(center + 2*sequence)$$

To get the  actual heuristic function, Manhattan Distance $P(n)$ will be added to Nilsson Score $3S(n)$. Note that the actual score is multiplied by 3. The original $S(n)$ is retained so that it can be printed out later in the program.

$$h_3(n) = P(n) + 3S(n)$$

$$f_3(n) = g(n) + h_3(n)$$
$f_3(n) =$ `cost + 3 * nscore + distance`

One move in any direction `up`, `down`, `left`, or `right` should have their own calculation $h_2(n)$.

# Final A* Algorithm Psuedocode

This does not build an actual tree-like structure instead uses a list called
`OPEN`, containing nodes ready for expansion, and another list called `CLOSED` containing expanded nodes.
1. Put the start node `s` on a list called `OPEN` and compute $f(s)$.
2. If `OPEN is empty()`, `exit` with failure; otherwise `continue`.
3. Remove from `OPEN` that node whose `f` value is smallest and put it on a list called `CLOSED`. Call this node `n`. (Resolve ties for minimal `f` values arbitrarily, but always in favor of any goal node.)
4. If `n` is a goal node, exit with the solution path obtained by tracing back the pointers; otherwise `continue`.
5. Expand node `n`, generating all of its successors. If there are no successors, go immediately to 2. For each successor `n_i`, compute $f(n_1)$.
6. Associate with the successors not already on either `OPEN` or `CLOSED` the `f` values just computed. Put these nodes on `OPEN` and direct pointers from them back to `n`.
7. Associate with those successors that were already on `OPEN` or `CLOSED` the smaller of the `f` values just computed and their previous `f` values. Put on `OPEN` those successors on `CLOSED` whose `f` values
were thus lowered, and redirect to `n` the pointers from all nodes whose `f` values were lowered.
8. Go to 2.

###Comments:
- duplicates are not retained; when nodes are rediscovered, the ancestor history is updated

#Implementing the Algorithm
To solve the 8-puzzle problem using the `start` and `goal` matrices parsed from `/content/astar_in.txt`, the following will be done:

1. **Heuristic Functions**:
    - `h1_misplaced`: Counts the number of tiles in the wrong position.
    - `h2_distance`: Calculates the sum of Manhattan distances for all tiles. This also calculates $P(n)$.
    - `h3_nilsson`: Computes Nilsson's Sequence Score (center tile score + clockwise sequence scores, multiplied by 3) and adds it to the Manhattan distance.
    - `get_nscore`: Computes the raw Nilsson score itself $S(n)$.
2. **A*** **Algorithm**: Implement using `OPEN` and `CLOSED` lists from the Pseudocode.
    - Expand nodes in the directions `up`, `down`, `left`, and `right`.
    - Handle state rediscovery by updating pointers and `f(n)` values if a shorter path to a node in `OPEN` or `CLOSED` is found.
    - Track the path using `parent` pointers.
3. **Output Formatting**: For each heuristic, display the solution path with the following:
    - The direction of the move (`UP`, `LEFT`, etc.).
    - The Algorithm's cost (number of nodes expanded)
    - The Depth of the node or total cost $g(n)$.
    - A visualization of the matrix (using '*' for the empty tile).

## Heuristic Functions

The three heuristic functions (`h1`, `h2`, `h3`) are defined below.
- `h1_misplaced(state, goal)` - will calculate the number of misplaced tiles of the state $n$
- `h2_distance(state, goal)` - will calculate the manhattan distance $P(n)$ of the state $n$
- `get_nscore(state, goal)` - will calculate the raw Nilsson score $N(n)$ of the state $n$
- `h3_nilsson(state, goal)` - will calculate the complete Nilsson score of the state $n$ using the two above functions

**Run** the code snippet to initialize the functions above.

In [182]:
def h1_misplaced(state, goal):
    count = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0 and state[i][j] != goal[i][j]:
                count += 1
    return count

def h2_distance(state, goal):
    def find_pos(matrix):
        pos = {}
        for r in range(3):
            for c in range(3):
                pos[matrix[r][c]] = (r, c)
        return pos

    current_pos = find_pos(state)
    target_pos = find_pos(goal)
    distance = 0
    for tile in range(1, 9):
        r1, c1 = current_pos[tile]
        r2, c2 = target_pos[tile]
        distance += abs(r1 - r2) + abs(c1 - c2)
    return distance

def get_nscore(state, goal):
    perimeter_indices = [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 1), (2, 0), (1, 0)]
    s_val = 0
    if state[1][1] != 0:
        s_val += 1
    p_state = [state[r][c] for r, c in perimeter_indices]
    p_goal = [goal[r][c] for r, c in perimeter_indices]
    for i in range(8):
        curr_tile = p_state[i]
        next_tile = p_state[(i + 1) % 8]
        try:
            idx_in_goal = p_goal.index(curr_tile)
            expected_next = p_goal[(idx_in_goal + 1) % 8]
            if next_tile != expected_next:
                s_val += 2
        except ValueError:
            s_val += 2
    return s_val

def h3_nilsson(state, goal):
    h2 = h2_distance(state, goal)
    nscore = get_nscore(state, goal)
    return h2 + 3 * nscore

# Testing with existing variables
print(f"h1 (Misplaced): {h1_misplaced(start, goal)}")
print(f"h2 (Manhattan): {h2_distance(start, goal)}")
print(f"Raw Nilsson Score (nscore): {get_nscore(start, goal)}")
print(f"h3 (Nilsson + Manhattan): {h3_nilsson(start, goal)}")

h1 (Misplaced): 7
h2 (Manhattan): 12
Raw Nilsson Score (nscore): 16
h3 (Nilsson + Manhattan): 60


## Implement A* Logic

From the heuristics and pseudocode, the **optimal path** will be found using any of the heuristics chosen. The following are initialized:
- `class Node` - with parameters such as the `state`, `scores`, `p` and `s` values, `parent`, and `move`.
- `get_neighbors(state)` - for step 5 of the psudocode that will generate all successors and checks if moves are valid (`up`,`down`,`left`, and `right`).
- `a_star_search(...)` - the main engine which sets up `OPEN` and `CLOSE` as a dictionary list to keep track of states and uses `get_neighbors()` to find new states.

  Note that `a_star_search(...)` updates itself to stay on the **most efficient path** as indicated by the pseudocode.

**Run** the code below to initialize the main class and functions.

In [183]:
import heapq #very fast instead of using a standard list to get min f value

class Node:
    def __init__(self, state, parent=None, move=None, g=0, h=0, move_idx=0, p_val=0, s_val=0):
        self.state = tuple(tuple(row) for row in state)
        self.parent = parent
        self.move = move
        self.g = g #cost only
        self.h = h #heuristic only
        self.f = g + h #total score
        self.move_idx = move_idx
        self.move_diff = 0 # Difference in expansion count from parent
        self.p_val = p_val # Manhattan Distance component
        self.s_val = s_val # Raw Nilsson Score component

    def __lt__(self, other):
        return self.f < other.f

def get_neighbors(state):
    neighbors = []
    r, c = -1, -1
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                r, c = i, j
                break
    moves = {"UP": (-1, 0), "DOWN": (1, 0), "LEFT": (0, -1), "RIGHT": (0, 1)}
    for move_name, (dr, dc) in moves.items():
        nr, nc = r + dr, c + dc
        if 0 <= nr < 3 and 0 <= nc < 3:
            new_state = [list(row) for row in state]
            new_state[r][c], new_state[nr][nc] = new_state[nr][nc], new_state[r][c]
            neighbors.append((new_state, move_name))
    return neighbors

def a_star_search(start_state, goal_state, heuristic_fn, h_type="h1"):
    expansion_counter = 0

    def get_h_data(s):
        if h_type == "h1": return heuristic_fn(s, goal_state), 0, 0
        if h_type == "h2":
            val = heuristic_fn(s, goal_state)
            return val, val, 0
        if h_type == "h3":
            p = h2_distance(s, goal_state)
            s_score = get_nscore(s, goal_state)
            return p + 3 * s_score, p, s_score
        return 0, 0, 0

    h_val, p_val, s_val = get_h_data(start_state)
    start_node = Node(start_state, None, None, 0, h_val, 0, p_val, s_val)
    goal_tuple = tuple(tuple(row) for row in goal_state)
    open_list = [start_node]
    closed_list = {}
    open_dict = {start_node.state: start_node.g}

    while open_list:
        current_node = heapq.heappop(open_list)
        if current_node.state in open_dict:
            del open_dict[current_node.state]

        current_node.move_idx = expansion_counter
        if current_node.parent:
            current_node.move_diff = current_node.move_idx - current_node.parent.move_idx
        else:
            current_node.move_diff = 0

        expansion_counter += 1

        if current_node.state == goal_tuple:
            return current_node, expansion_counter

        closed_list[current_node.state] = current_node.g

        for next_state_list, move in get_neighbors([list(r) for r in current_node.state]):
            next_state = tuple(tuple(r) for r in next_state_list)
            new_g = current_node.g + 1
            if next_state in closed_list and new_g >= closed_list[next_state]:
                continue
            if next_state in open_dict and new_g >= open_dict[next_state]:
                continue

            h, p, s = get_h_data(next_state_list)
            successor = Node(next_state_list, current_node, move, new_g, h, 0, p, s)
            heapq.heappush(open_list, successor)
            open_dict[next_state] = new_g
            if next_state in closed_list:
                del closed_list[next_state]
    return None, expansion_counter

print("A* logic implemented with move_diff support")

A* logic implemented with move_diff support


## Final Printing Format
The following functions are implemented to construct the final solutions:
- `path_final(goal_node)` - this backtracks and reconstructs the `path` once `goal` is achieved. Starts with the `goal_node`.
- `print_solution(path, h_type)` - this is the main visualizer that will print the `path` until the `goal`.
- `print_table(path)` - this will tabulate results.
- `full_solve(...)` - this will show the whole solution including the `path` and summation of expanded nodes on each step.
- `quick_solve(...)` - this will show the final results only including the total number of expanded nodes.

**Run** the code below to initialize the formatting functions.


In [231]:
def path_final(goal_node):
    path = []
    current = goal_node
    while current:
        path.append(current)
        current = current.parent
    return path[::-1] #go backwards!

def print_solution(path, h_type):
    for node in path:
        move_label = node.move if node.move else "START"
        mat_str = "["
        for r, row in enumerate(node.state):
            line = "[" + " ".join(str(val) if val != 0 else "*" for val in row) + "]"
            mat_str += line + ("\n " if r < 2 else "")
        mat_str += "]"

        print(f"Move {node.g}: {move_label}")
        print(mat_str)
        print(f"Nodes Expanded = {node.move_idx} (added {node.move_diff})")
        print(f"g(n) = {node.g}")
        print(f"h(n) = {node.h}")

        if h_type == "h2":
            print(f"P(n) = {node.p_val}")
        elif h_type == "h3":
            print(f"P(n) = {node.p_val}")
            print(f"S(n) = {node.s_val}")
        print("-" * 10)

def print_table(path):
    # Re-run all searches to get their specific expansion timings
    res1, _ = a_star_search(start, goal, h1_misplaced, "h1")
    res2, _ = a_star_search(start, goal, h2_distance, "h2")
    res3, _ = a_star_search(start, goal, h3_nilsson, "h3")

    p1 = path_final(res1)
    p2 = path_final(res2)
    p3 = path_final(res3)

    # Map depth to expansion count for each path
    e1_map = {n.g: n.move_idx for n in p1}
    e2_map = {n.g: n.move_idx for n in p2}
    e3_map = {n.g: n.move_idx for n in p3}

    print(f"\nTabulated Results:")
    header = "{:<3} | {:<4} | {:<4} | {:<4} | {:<4} | {:<4} | {:<4} | {:<4} | {:<4} | {:<4}".format(
        "i", "g(n)", "h1(n)", "E1(n)", "h2(n)", "E2(n)", "h3(n)", "E3(n)", "P(n)", "S(n)"
    )
    print(header)
    print("-" * len(header))

    # Use the provided path (usually the most efficient one) as the baseline for states
    for i, node in enumerate(path):
        state_list = [list(r) for r in node.state]
        h1 = h1_misplaced(state_list, goal)
        h2 = h2_distance(state_list, goal)
        h3 = h3_nilsson(state_list, goal)
        s_val = get_nscore(state_list, goal)

        # Get expansion counts for this depth from each heuristic run
        e1 = e1_map.get(node.g, "-")
        e2 = e2_map.get(node.g, "-")
        e3 = e3_map.get(node.g, "-")

        row = "{:<3} | {:<4} | {:<4} | {:<4} | {:<4} | {:<4} | {:<4} | {:<4} | {:<4} | {:<4}".format(
            i, node.g, h1, e1, h2, e2, h3, e3, h2, s_val
        )
        print(row)

def full_solve(start_state, goal_state, heuristic_fn, label, h_type):
    print(f"\n{'='*40}\n{label}\n{'='*40}")
    result_node, total_expansions = a_star_search(start_state, goal_state, heuristic_fn, h_type)
    if result_node:
        path = path_final(result_node)
        print_solution(path, h_type)
        print(f"{label} FINAL RESULTS")
        print(f"Total Nodes Expanded: {total_expansions}")
        print(f"Total Number of Moves: {result_node.g}")
        return path
    else:
        print("No solution found.")
        return None

def quick_solve(start_state, goal_state, heuristic_fn, label, h_type):
    print(f"\n{'='*40}\n{label}\n{'='*40}")
    result_node, total_expansions = a_star_search(start_state, goal_state, heuristic_fn, h_type)
    if result_node:
        print(f"Total Nodes Expanded: {total_expansions}")
        print(f"Total Number of Moves: {result_node.g}")
        return path_final(result_node)
    else:
        print("No solution found.")
        return None

# Final Results

In this section, results for all methods used will be printed using the final printing format.

**Run** each method below to see the results.
**Expand** each header to see:
- **Quick Solution** - to see only final results.
- **Full Solution** - to see the whole `path` including number of expanded nodes and depth or cost.


## a) Misplaced Tiles

In [192]:
quick_solve(start, goal, h1_misplaced, "Misplaced Tiles", "h1")


Misplaced Tiles
Total Nodes Expanded: 1487
Total Number of Moves: 18


### Full Solution
Expand to see path when using **Misplaced Tiles** heuristic.

In [193]:
full_solve(start, goal, h1_misplaced, "Misplaced Tiles", "h1")


Misplaced Tiles
Move 0: START
[[2 1 6]
 [4 * 8]
 [7 5 3]]
Nodes Expanded = 0 (added 0)
g(n) = 0
h(n) = 7
----------
Move 1: UP
[[2 * 6]
 [4 1 8]
 [7 5 3]]
Nodes Expanded = 1 (added 1)
g(n) = 1
h(n) = 7
----------
Move 2: LEFT
[[* 2 6]
 [4 1 8]
 [7 5 3]]
Nodes Expanded = 5 (added 4)
g(n) = 2
h(n) = 6
----------
Move 3: DOWN
[[4 2 6]
 [* 1 8]
 [7 5 3]]
Nodes Expanded = 8 (added 3)
g(n) = 3
h(n) = 6
----------
Move 4: RIGHT
[[4 2 6]
 [1 * 8]
 [7 5 3]]
Nodes Expanded = 22 (added 14)
g(n) = 4
h(n) = 6
----------
Move 5: RIGHT
[[4 2 6]
 [1 8 *]
 [7 5 3]]
Nodes Expanded = 27 (added 5)
g(n) = 5
h(n) = 6
----------
Move 6: UP
[[4 2 *]
 [1 8 6]
 [7 5 3]]
Nodes Expanded = 67 (added 40)
g(n) = 6
h(n) = 6
----------
Move 7: LEFT
[[4 * 2]
 [1 8 6]
 [7 5 3]]
Nodes Expanded = 210 (added 143)
g(n) = 7
h(n) = 7
----------
Move 8: LEFT
[[* 4 2]
 [1 8 6]
 [7 5 3]]
Nodes Expanded = 315 (added 105)
g(n) = 8
h(n) = 7
----------
Move 9: DOWN
[[1 4 2]
 [* 8 6]
 [7 5 3]]
Nodes Expanded = 317 (added 2)
g(n) = 9

## b) Manhattan Distance

In [194]:
quick_solve(start, goal, h2_distance, "Manhattan Distance", "h2")



Manhattan Distance
Total Nodes Expanded: 165
Total Number of Moves: 18


### 2. Full Solution

Expand to see path when using **Manhattan Distance** heuristic.

In [195]:
full_solve(start, goal, h2_distance, "Manhattan Distance", "h2")


Manhattan Distance
Move 0: START
[[2 1 6]
 [4 * 8]
 [7 5 3]]
Nodes Expanded = 0 (added 0)
g(n) = 0
h(n) = 12
P(n) = 12
----------
Move 1: UP
[[2 * 6]
 [4 1 8]
 [7 5 3]]
Nodes Expanded = 11 (added 11)
g(n) = 1
h(n) = 13
P(n) = 13
----------
Move 2: LEFT
[[* 2 6]
 [4 1 8]
 [7 5 3]]
Nodes Expanded = 14 (added 3)
g(n) = 2
h(n) = 12
P(n) = 12
----------
Move 3: DOWN
[[4 2 6]
 [* 1 8]
 [7 5 3]]
Nodes Expanded = 55 (added 41)
g(n) = 3
h(n) = 13
P(n) = 13
----------
Move 4: RIGHT
[[4 2 6]
 [1 * 8]
 [7 5 3]]
Nodes Expanded = 58 (added 3)
g(n) = 4
h(n) = 12
P(n) = 12
----------
Move 5: RIGHT
[[4 2 6]
 [1 8 *]
 [7 5 3]]
Nodes Expanded = 60 (added 2)
g(n) = 5
h(n) = 11
P(n) = 11
----------
Move 6: UP
[[4 2 *]
 [1 8 6]
 [7 5 3]]
Nodes Expanded = 62 (added 2)
g(n) = 6
h(n) = 10
P(n) = 10
----------
Move 7: LEFT
[[4 * 2]
 [1 8 6]
 [7 5 3]]
Nodes Expanded = 81 (added 19)
g(n) = 7
h(n) = 11
P(n) = 11
----------
Move 8: LEFT
[[* 4 2]
 [1 8 6]
 [7 5 3]]
Nodes Expanded = 88 (added 7)
g(n) = 8
h(n) = 10
P

## c) Nilsson Sequence Score

In [196]:
quick_solve(start, goal, h3_nilsson, "Nilsson Sequence Score", "h3")


Nilsson Sequence Score
Total Nodes Expanded: 45
Total Number of Moves: 18


### Full Solution

This is the most efficient method by far.

Expand to see path when using **Nilsson Sequence Score** heuristic.

In [204]:
full_solve(start, goal, h3_nilsson, "Nilsson Sequence Score", "h3")


Nilsson Sequence Score
Move 0: START
[[2 1 6]
 [4 * 8]
 [7 5 3]]
Nodes Expanded = 0 (added 0)
g(n) = 0
h(n) = 60
P(n) = 12
S(n) = 16
----------
Move 1: RIGHT
[[2 1 6]
 [4 8 *]
 [7 5 3]]
Nodes Expanded = 2 (added 2)
g(n) = 1
h(n) = 62
P(n) = 11
S(n) = 17
----------
Move 2: UP
[[2 1 *]
 [4 8 6]
 [7 5 3]]
Nodes Expanded = 3 (added 1)
g(n) = 2
h(n) = 61
P(n) = 10
S(n) = 17
----------
Move 3: LEFT
[[2 * 1]
 [4 8 6]
 [7 5 3]]
Nodes Expanded = 8 (added 5)
g(n) = 3
h(n) = 62
P(n) = 11
S(n) = 17
----------
Move 4: DOWN
[[2 8 1]
 [4 * 6]
 [7 5 3]]
Nodes Expanded = 9 (added 1)
g(n) = 4
h(n) = 54
P(n) = 12
S(n) = 14
----------
Move 5: RIGHT
[[2 8 1]
 [4 6 *]
 [7 5 3]]
Nodes Expanded = 11 (added 2)
g(n) = 5
h(n) = 56
P(n) = 11
S(n) = 15
----------
Move 6: DOWN
[[2 8 1]
 [4 6 3]
 [7 5 *]]
Nodes Expanded = 12 (added 1)
g(n) = 6
h(n) = 55
P(n) = 10
S(n) = 15
----------
Move 7: LEFT
[[2 8 1]
 [4 6 3]
 [7 * 5]]
Nodes Expanded = 13 (added 1)
g(n) = 7
h(n) = 54
P(n) = 9
S(n) = 15
----------
Move 8: UP
[[

## All Methods Combined

In [229]:
path = full_solve(start, goal, h1_misplaced, "Misplaced Tiles", "h1")
path = full_solve(start, goal, h2_distance, "Manhattan Distance", "h2")
path = full_solve(start, goal, h3_nilsson, "Nilsson Sequence Score", "h3")

#print_table(path)


Misplaced Tiles
Move 0: START
[[2 1 6]
 [4 * 8]
 [7 5 3]]
Nodes Expanded = 0 (added 0)
g(n) = 0
h(n) = 7
----------
Move 1: UP
[[2 * 6]
 [4 1 8]
 [7 5 3]]
Nodes Expanded = 1 (added 1)
g(n) = 1
h(n) = 7
----------
Move 2: LEFT
[[* 2 6]
 [4 1 8]
 [7 5 3]]
Nodes Expanded = 5 (added 4)
g(n) = 2
h(n) = 6
----------
Move 3: DOWN
[[4 2 6]
 [* 1 8]
 [7 5 3]]
Nodes Expanded = 8 (added 3)
g(n) = 3
h(n) = 6
----------
Move 4: RIGHT
[[4 2 6]
 [1 * 8]
 [7 5 3]]
Nodes Expanded = 22 (added 14)
g(n) = 4
h(n) = 6
----------
Move 5: RIGHT
[[4 2 6]
 [1 8 *]
 [7 5 3]]
Nodes Expanded = 27 (added 5)
g(n) = 5
h(n) = 6
----------
Move 6: UP
[[4 2 *]
 [1 8 6]
 [7 5 3]]
Nodes Expanded = 67 (added 40)
g(n) = 6
h(n) = 6
----------
Move 7: LEFT
[[4 * 2]
 [1 8 6]
 [7 5 3]]
Nodes Expanded = 210 (added 143)
g(n) = 7
h(n) = 7
----------
Move 8: LEFT
[[* 4 2]
 [1 8 6]
 [7 5 3]]
Nodes Expanded = 315 (added 105)
g(n) = 8
h(n) = 7
----------
Move 9: DOWN
[[1 4 2]
 [* 8 6]
 [7 5 3]]
Nodes Expanded = 317 (added 2)
g(n) = 9

## Tabulate Results
- `g(n)` - the cost / number of moves
- `h(n)` - the heuristic function used
- `E(n)` - the number of expanded nodes

**Run** the code below to show the table's final results.

In [233]:
print_table(path)


Tabulated Results:
i   | g(n) | h1   | E1   | h2   | E2   | h3   | E3   | P(n) | S(n)
------------------------------------------------------------------
0   | 0    | 7    | 0    | 12   | 0    | 60   | 0    | 12   | 16  
1   | 1    | 7    | 1    | 11   | 11   | 62   | 2    | 11   | 17  
2   | 2    | 7    | 5    | 10   | 14   | 61   | 3    | 10   | 17  
3   | 3    | 7    | 8    | 11   | 55   | 62   | 8    | 11   | 17  
4   | 4    | 7    | 22   | 12   | 58   | 54   | 9    | 12   | 14  
5   | 5    | 7    | 27   | 11   | 60   | 56   | 11   | 11   | 15  
6   | 6    | 7    | 67   | 10   | 62   | 55   | 12   | 10   | 15  
7   | 7    | 6    | 210  | 9    | 81   | 54   | 13   | 9    | 15  
8   | 8    | 5    | 315  | 8    | 88   | 38   | 14   | 8    | 10  
9   | 9    | 5    | 317  | 7    | 94   | 40   | 15   | 7    | 11  
10  | 10   | 5    | 319  | 8    | 99   | 41   | 17   | 8    | 11  
11  | 11   | 5    | 550  | 7    | 106  | 46   | 37   | 7    | 13  
12  | 12   | 5    | 921  | 6    | 113  | 3